# Linear models  

In this notebook we will be looking at ways to use linear models to predict electricity demand for the GTA. We will trying different combinations of features through best subset selection which in the end should give us a sense of the most important features.

In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.linear_model
import math

In [2]:
# Load the data (Also removing columns that contain repetitive information)
data = pd.read_csv("MergedDataset.csv", delimiter = ",").drop(['time', 'local_time'], axis=1)
data

C:\Users\Greta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date,Hour,Toronto,HOEP,temperature,precipitation,snowfall,snow_mass,air_density,radiation_surface,radiation_toa,cloud_cover,isWeekend,isHoliday
0,2004-01-01,1,4606,30.9,0.198,0.001,0.000,1.156,1.279,0.0,0.0,0.118,False,True
1,2004-01-01,2,4366,27.13,0.339,0.001,0.000,1.156,1.279,0.0,0.0,0.148,False,True
2,2004-01-01,3,4188,25.23,0.502,0.001,0.001,1.156,1.280,0.0,0.0,0.144,False,True
3,2004-01-01,4,4046,24.29,0.534,0.000,0.000,1.157,1.280,0.0,0.0,0.159,False,True
4,2004-01-01,5,3974,24.42,0.494,0.000,0.000,1.157,1.281,0.0,0.0,0.194,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131491,2018-12-31,20,5755,5.62,2.908,2.279,0.566,3.426,1.243,0.0,0.0,0.988,False,False
131492,2018-12-31,21,5544,8.95,2.383,1.402,0.263,3.773,1.242,0.0,0.0,0.981,False,False
131493,2018-12-31,22,5338,5.81,2.611,0.244,0.041,3.866,1.237,0.0,0.0,0.985,False,False
131494,2018-12-31,23,5091,2.87,3.384,0.283,0.076,3.872,1.227,0.0,0.0,0.990,False,False


In [3]:
#Checking the types of each column of our loaded data.
data.dtypes

Date                  object
Hour                   int64
Toronto                int64
HOEP                  object
temperature          float64
precipitation        float64
snowfall             float64
snow_mass            float64
air_density          float64
radiation_surface    float64
radiation_toa        float64
cloud_cover          float64
isWeekend               bool
isHoliday               bool
dtype: object

In [4]:
#Converting the 'Date' column from object to datetime.
data['Date'] = pd.to_datetime(data['Date'])

In [5]:
#Checking that our 'Date' column got converted to the right type
data.dtypes

Date                 datetime64[ns]
Hour                          int64
Toronto                       int64
HOEP                         object
temperature                 float64
precipitation               float64
snowfall                    float64
snow_mass                   float64
air_density                 float64
radiation_surface           float64
radiation_toa               float64
cloud_cover                 float64
isWeekend                      bool
isHoliday                      bool
dtype: object

In [6]:
#Updating whole data set to have 7 years studied total(5 years training data + 2 years training data)
data = data[data['Date'].dt.year >= 2012]
data

,Date,Hour,Toronto,HOEP,temperature,precipitation,snowfall,snow_mass,air_density,radiation_surface,radiation_toa,cloud_cover,isWeekend,isHoliday
70128,2012-01-01,1,4834,21.71,0.672,0.032,0.004,6.048,1.267,0.0,0.0,0.538,True,True
70129,2012-01-01,2,4631,16.92,0.932,0.004,0.002,6.054,1.265,0.0,0.0,0.581,True,True
70130,2012-01-01,3,4442,11.23,1.319,0.006,0.004,6.059,1.262,0.0,0.0,0.676,True,True
70131,2012-01-01,4,4312,7.1,1.715,0.008,0.004,6.066,1.259,0.0,0.0,0.624,True,True
70132,2012-01-01,5,4224,-6.7,2.019,0.013,0.003,6.070,1.256,0.0,0.0,0.637,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131491,2018-12-31,20,5755,5.62,2.908,2.279,0.566,3.426,1.243,0.0,0.0,0.988,False,False
131492,2018-12-31,21,5544,8.95,2.383,1.402,0.263,3.773,1.242,0.0,0.0,0.981,False,False
131493,2018-12-31,22,5338,5.81,2.611,0.244,0.041,3.866,1.237,0.0,0.0,0.985,False,False
131494,2018-12-31,23,5091,2.87,3.384,0.283,0.076,3.872,1.227,0.0,0.0,0.990,False,False


In [7]:
# Create a new column 't' which is just the index of each date starting at 1
data.insert(0, 't', np.arange(1, len(data)+1)) 

#Checking that the new column gets created and that it does what it's expected to
data

,t,Date,Hour,Toronto,HOEP,temperature,precipitation,snowfall,snow_mass,air_density,radiation_surface,radiation_toa,cloud_cover,isWeekend,isHoliday
70128,1,2012-01-01,1,4834,21.71,0.672,0.032,0.004,6.048,1.267,0.0,0.0,0.538,True,True
70129,2,2012-01-01,2,4631,16.92,0.932,0.004,0.002,6.054,1.265,0.0,0.0,0.581,True,True
70130,3,2012-01-01,3,4442,11.23,1.319,0.006,0.004,6.059,1.262,0.0,0.0,0.676,True,True
70131,4,2012-01-01,4,4312,7.1,1.715,0.008,0.004,6.066,1.259,0.0,0.0,0.624,True,True
70132,5,2012-01-01,5,4224,-6.7,2.019,0.013,0.003,6.070,1.256,0.0,0.0,0.637,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131491,61364,2018-12-31,20,5755,5.62,2.908,2.279,0.566,3.426,1.243,0.0,0.0,0.988,False,False
131492,61365,2018-12-31,21,5544,8.95,2.383,1.402,0.263,3.773,1.242,0.0,0.0,0.981,False,False
131493,61366,2018-12-31,22,5338,5.81,2.611,0.244,0.041,3.866,1.237,0.0,0.0,0.985,False,False
131494,61367,2018-12-31,23,5091,2.87,3.384,0.283,0.076,3.872,1.227,0.0,0.0,0.990,False,False


In [14]:
#Creating our train and test set 
data_train = data[data['Date'].dt.year < 2017]
data_test = data[data['Date'].dt.year >= 2017]
data_train

,t,Date,Hour,Toronto,HOEP,temperature,precipitation,snowfall,snow_mass,air_density,radiation_surface,radiation_toa,cloud_cover,isWeekend,isHoliday
70128,1,2012-01-01,1,4834,21.71,0.672,0.032,0.004,6.048,1.267,0.0,0.0,0.538,True,True
70129,2,2012-01-01,2,4631,16.92,0.932,0.004,0.002,6.054,1.265,0.0,0.0,0.581,True,True
70130,3,2012-01-01,3,4442,11.23,1.319,0.006,0.004,6.059,1.262,0.0,0.0,0.676,True,True
70131,4,2012-01-01,4,4312,7.1,1.715,0.008,0.004,6.066,1.259,0.0,0.0,0.624,True,True
70132,5,2012-01-01,5,4224,-6.7,2.019,0.013,0.003,6.070,1.256,0.0,0.0,0.637,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113971,43844,2016-12-31,20,5665,2.81,2.506,0.140,0.014,4.203,1.243,0.0,0.0,0.875,True,False
113972,43845,2016-12-31,21,5665,0.0,2.218,0.043,0.002,4.131,1.245,0.0,0.0,0.579,True,False
113973,43846,2016-12-31,22,5665,0.0,1.579,0.017,0.005,4.099,1.250,0.0,0.0,0.188,True,False
113974,43847,2016-12-31,23,5665,0.0,0.515,0.007,0.003,4.101,1.258,0.0,0.0,0.172,True,False


To account for the seasonality in the data it will not be enough to consider a linear model of the form $X_t = \beta_1 t + \beta_0$ as this would only pickup on the underlying linear trend in the data (for which we already suspect is relatively weak according to our intial plots and time series decomposition <b>Note to self: this may be subject to change depending on how the time series analysis goes</b>).

We need to introduce some seasonal behaviour in our linear model. One way to do this (<b>CITE HERE</b>) is through fourier feature bases. If $m$ is the seasonal period we suspect (e.g. daily, weekly, monthly, yearly) then we can introduce terms of the following form:
$X_t = \sum_{j=1}^P \beta_{2j-1} \sin(\frac{2 \pi j t}{m}) + \beta_{2j} \cos(\frac{2 \pi j t}{m})$
where $m$ is the the seasonal period and $P$ is the number of pairs of fourier series we have. We will bound $P$ to at most $\frac{m}{2}$. For daily seasonality $m=24$, for weekly $m=7 \times 24$, for monthly $m = 4 \times 7 \times 24$ (using the running assumption that we use a month as 4 weeks) and yearly as $365 \times 24 = 8760$.


In [9]:
#Creating our design matrix (adding t to it)
design_matrix = np.array(data_train['t']).reshape(-1,1)

#Daily
m = 24
#MAKE A COMMENT
j = np.arange(1, int(m/2)+1)
#np.array(range(1, int(m/2)))
titles_sin, titles_cos = ['daily_sin_' + str(x) for x in j], ['daily_cos_' + str(x) for x in j]
temp_multiply = (2 * np.pi * j)/m
daily_design_matrix_sin = np.sin(temp_multiply * design_matrix)
daily_design_matrix_cos = np.cos(temp_multiply * design_matrix)
# for j in range(1, int(m/2)):
#     design_matrix['daily_sin_' + str(j)] = design_matrix.apply(lambda row: math.sin((2 * math.pi * j * row.t) / m), axis=1)
#     design_matrix['daily_cos_' + str(j)] = design_matrix.apply(lambda row: math.cos((2 * math.pi * j * row.t) / m), axis=1)

#Weekly
#m = 24 * 7
#MAKE A COMMENT
#j = np.array([i//2 + 1 for i in range(0, m)])
#titles_sin_cos = [('weekly_sin_' + str(x), 'weekly_cos_' + str(x)) for x in j]
#titles_sin_cos = list(sum(titles_sin_cos, ()))
#temp_multiply = (2 * np.pi * j)/m
#weekly_design_matrix = temp_multiply * design_matrix
# for j in range(1, int(m/2)):
#     design_matrix['weekly_sin' + str(j)] = design_matrix.apply(lambda row: math.sin((2 * math.pi * row.t) / m), axis=1)
#     design_matrix['weekly_cos' + str(j)] = design_matrix.apply(lambda row: math.cos((2 * math.pi * row.t) / m), axis=1)

#Yearly
#m = 365 * 24
#MAKE A COMMENT
#j = np.array([i//2 + 1 for i in range(0, m)])
#titles_sin_cos = [('yearly_sin_' + str(x), 'yearly_cos_' + str(x)) for x in j]
#titles_sin_cos = list(sum(titles_sin_cos, ()))
#temp_multiply = (2 * np.pi * j)/m
#yearly_design_matrix = temp_multiply * design_matrix
#for j in range(1, int(m/2)):
#    design_matrix['yearly_sin' + str(j)] = design_matrix.apply(lambda row: math.sin((2 * math.pi * row.t) / m), axis=1)
#    design_matrix['yearly_cos' + str(j)] = design_matrix.apply(lambda row: math.cos((2 * math.pi * row.t) / m), axis=1)

In [10]:
design_matrix

array([[    1],
       [    2],
       [    3],
       ...,
       [43846],
       [43847],
       [43848]])

In [11]:
# Add columns for each of the fourier bases from j = 1 to P and for each different seasonality
# for j = 1 to m/2, but do this in ranges:
# for m = 24 use the range [1,...12]
# For m = 7*24 use a longer range like [1, 6, 12, ..., P] (so we don't blow up training)
# For m = 4*7*42 use a longer range [1, 12, ... P] (remember P = m/2 use P - 1 to be safe actually)
# Again the same thing for m = 8760 use a larger ranger something divisible by 8760 like 10, 20

In [12]:
# For each possible pair train the model with cross validation with the following function
#The model is the sklearn model, the n_splits is the number of folds (try to find a reasonable
# size but i suspect we won't be able to do it a large number of splits, there's just
# too much data! )
# X is the transformed set of features and y is the target electricity demand
def rolling_window_time_series(model, n_splits, X, y):
    # Keep a running total of the mean mse
    mse_cv = 0
    N = 1
    ts_splitter = sklearn.model_selection.TimeSeriesSplit(n_splits=n_splits)
    for train, valid in ts_splitter.split(X):
        # For each train, valid fold fit it on train
        model.fit(X[train], y[train])
        # Predict on valid
        y_pred = model.predict(X[valid])
        # Find mse of this fold and then add it to the running average
        mse_fold = sklearn.metrics.mean_squared_error(y_true=y[valid], y_pred=y_pred)
        # This is to keep a running average (instead of appending to an array and then
        # taking the mean)
        if N == 1:
            mse_cv = mse_fold
        else:
            # This can be checked by expanding the formula for the mean
            mse_cv += (mse_fold - mse_cv)/N
        N+=1
    # Return mse and rmse
    return mse_cv, math.sqrt(mse_cv)

# 1. So high level it's like a for loop over all possible combiantions of parameters
# 2. For each of those combinations train the model using cross validation
# Record the average mse => 2. should be done by the function rolling_window

In [13]:
# Once you get the optimal fourier series linear_model
# Do best subset selection on the 10 features so 2^10 combinations now added to the 
# best model you found above